Task
1. find travel time and then change it to hours
2. Find who travel between 1 to 60 hr
3. Train model 
        - LinearRegression
        - Lasso
        

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle as pic

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.metrics import mean_squared_error

In [ ]:

df=pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")
df

In [ ]:
df=df[df.trip_type==2]
df

In [ ]:
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])
df["duration"]=df.lpep_dropoff_datetime-df.lpep_pickup_datetime
df.duration=df.duration.apply(lambda td: td.total_seconds()/60)
df=df[(df.duration>=1) & (df.duration<=60)]
categorical=['PULocationID','DOLocationID']
numerical=['trip_distance']
df[categorical]=df[categorical].astype(str)


In [ ]:
train_dict=df[categorical+numerical].to_dict(orient='records')
dv=DictVectorizer()
X_train=dv.fit_transform(train_dict)
target='duration'
y_train=df[target].values
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_train)
mean_squared_error(y_train,y_pred,squared=False)

In [ ]:
sns.displot(y_pred,label='prediction')
sns.displot(y_train,label='actual')
plt.legend()

In [ ]:
#df=df[df.trip_type==2]
#df

In [ ]:
sns.distplot(df.duration)

In [ ]:
df.duration.describe(percentiles=[0.05,0.98,0.99])

In [ ]:
df_train['PU_DO']=df_train['PULocationID']+'_'+df_train['DOLocationID']
df_val['PU_DO']=df_val['PULocationID']+'_'+df_val['DOLocationID']
df_val

In [ ]:
def read_dataframe(filename):
    df=pd.read_parquet(filename)
    df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
    df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])
    df["duration"]=df.lpep_dropoff_datetime-df.lpep_pickup_datetime
    df.duration=df.duration.apply(lambda td: td.total_seconds()/60)
    df=df[(df.duration>=1) & (df.duration<=60)]
    categorical=['PULocationID','DOLocationID']
    #categorical=['PU_DO']
    numerical=['trip_distance']
    df[categorical]=df[categorical].astype(str)
    return df


In [ ]:
df_train=read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')
df_val=read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
len(df_train) , len(df_val)

In [ ]:
dv=DictVectorizer()

#train
train_dict=df[categorical+numerical].to_dict(orient='records')
X_train=dv.fit_transform(train_dict)

#validation
val_dicts=df_val[categorical+numerical].to_dict(orient='records')
X_val=dv.transform(val_dicts)


#what we need to find
target='duration'
y_train=df[target].values
y_val=df_val[target].values



In [ ]:
#linear Regression
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_val)
mean_squared_error(y_val,y_pred,squared=False)

In [ ]:
#lasso
lr = Lasso(alpha=0.1)
lr.fit(X_train,y_train)
y_pred=lr.predict(X_val)
mean_squared_error(y_val,y_pred,squared=False)

In [ ]:
#Ridge
lr = Ridge(alpha=1)
lr.fit(X_train,y_train)
y_pred=lr.predict(X_val)
mean_squared_error(y_val,y_pred,squared=False)

In [ ]:
#save model
with open('models','wb') as f_out:
    pic.dump((dv,lr),f_out)

In [ ]:
td=df.duration.iloc[0]
td.total_seconds()/60